In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np

#### gs://marketplace-2xim6sjc/MedMNIST/120/artifactFiles/chestmnist_128.npz

# Initialize a client
client = storage.Client()

# Define the bucket and blob (file) names
bucket_name = 'marketplace-2xim6sjc'
blob_name = 'MedMNIST/120/artifactFiles/chestmnist_128.npz'

# Get the bucket and blob
bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)

# Download the blob to a local file
local_file_path = 'chestmnist_128_local.npz'
blob.download_to_filename(local_file_path)

# Load the .npz file
data = np.load(local_file_path, allow_pickle=True)

# Access the data
for key in data:
    print(f"{key}: {data[key]}")

train_images: [[[200 127  47 ...   9   8   6]
  [172  90  25 ...  15   8   7]
  [164  85  22 ...  15   8   6]
  ...
  [190  94  23 ... 154 139 120]
  [196  97  28 ... 173 162 145]
  [202 103  37 ... 193 184 169]]

 [[  3   3   3 ...   1   5  23]
  [  3   3   3 ...   1   5  23]
  [  3   3   3 ...   1   5  22]
  ...
  [  2   2   2 ...   1   3  14]
  [  3   3   3 ...   2   6  20]
  [  7   7   7 ...   6  12  32]]

 [[255 253 251 ... 242 245 241]
  [248 243 236 ... 232 236 233]
  [230 205 179 ... 215 220 219]
  ...
  [ 41  30  28 ...  28  28  30]
  [ 41  30  28 ...  28  28  30]
  [ 41  29  28 ...  28  28  30]]

 ...

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[ 25   6   0 ...  53  55  57]
  [ 22   3   0 ...  11  12  13]
  [ 20   3   0 ...   0   0   0]
  ...
  [ 42  52  66 ...   0   0   0]
  [ 41  52  66 ...   0   0   0]
  [ 43  55  69

In [2]:
!pip install google-cloud-storage numpy torch torchvision matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
# Extract the data
x_train = data['train_images']
y_train = data['train_labels']
x_val = data['val_images']
y_val = data['val_labels']
x_test = data['test_images']
y_test = data['test_labels']

# Normalize the images
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Convert to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train = torch.tensor(y_train, dtype=torch.long)
x_val = torch.tensor(x_val, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_val = torch.tensor(y_val, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 14 * 14, 128)
        self.fc2 = nn.Linear(128, 10)  # Assuming 10 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 14 * 14)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)